In [1]:
import pandas as pd

In [2]:
# read in original data.  this data contains cumulative cases per texas county per day
# a little extra provisions necessary to skip extraneous rows and to make columns what we want them to be

data_file_df = pd.read_excel('Texas COVID-19 Case Count Data by County.xlsx', skiprows={0, 1, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267})

# All along, we knew that this will need adjustment come October
# Five-digit month/years don't seem to easily fit into this method
# data_file_df.rename (columns=lambda x: x[4:], inplace=True)   

cols = data_file_df.columns.values
newcols = []
colcount = 0

for col in cols:
    colcount += 1
    if col == 'County Name':
        newcols.append(col)
    else:
        newcol = ""

        for char in col:
            if (char >= "0" and char <= "9") or char == "-":
                newcol = newcol + char
        
        print('/' + newcol + "/")
        newcols.append(newcol)

data_file_df.columns = newcols

# data for 3-07 and 3-08 are missing - so we'll pretend that any data before that is missing as well.
# these are mostly zero anyway.
del data_file_df["03-04-2020"]
del data_file_df["03-05-2020"]
del data_file_df["03-06-2020"]




/03-04-2020/
/03-05-2020/
/03-06-2020/
/03-09-2020/
/03-10-2020/
/03-11-2020/
/03-12-2020/
/03-13-2020/
/03-15-2020/
/03-16-2020/
/03-17-2020/
/03-18-2020/
/03-19-2020/
/03-20-2020/
/03-21-2020/
/03-22-2020/
/03-23-2020/
/03-24-2020/
/03-25-2020/
/03-26-2020/
/03-27-2020/
/03-28-2020/
/03-29-2020/
/03-30-2020/
/03-31-2020/
/04-01-2020/
/04-02-2020/
/04-03-2020/
/04-04-2020/
/04-05-2020/
/04-06-2020/
/04-07-2020/
/04-08-2020/
/04-09-2020/
/04-10-2020/
/04-11-2020/
/04-12-2020/
/04-13-2020/
/04-14-2020/
/04-15-2020/
/04-16-2020/
/04-17-2020/
/04-18-2020/
/04-19-2020/
/04-20-2020/
/04-21-2020/
/04-22-2020/
/04-23-2020/
/04-24-2020/
/04-25-2020/
/04-26-2020/
/04-27-2020/
/04-28-2020/
/04-29-2020/
/04-30-2020/
/05-01-2020/
/05-02-2020/
/05-03-2020/
/05-04-2020/
/05-05-2020/
/05-06-2020/
/05-07-2020/
/05-08-2020/
/05-09-2020/
/05-10-2020/
/05-11-2020/
/05-12-2020/
/05-13-2020/
/05-14-2020/
/05-15-2020/
/05-16-2020/
/05-17-2020/
/05-18-2020/
/05-19-2020/
/05-20-2020/
/05-21-2020/
/05-22-2020/

In [3]:
def assign_grade(value):

    
# original binning values:   20  10  8  6  4  2  >0  0  <0
#          labels:           A   B   C  D  E  F  G       N

    grade = " "
    if value >= 40:
        grade = ">40"
    elif value >= 28:
        grade = ">28"
    elif value >= 21:
        grade = ">21"
    elif value >= 15:
        grade = ">15"
    elif value >= 10:
        grade = ">10"
    elif value >= 5:
        grade = ">5"
    elif value > 0:
        grade = ">0"
    elif value == 0:
        grade = "none"
    elif value < 0:
        grade = "negative"
        
    return (grade);


In [4]:
# Additional county information not present in main data file

codes_df = pd.read_csv('codes2.csv')
codes_df.head()

,County Name,Section,Number,Population,FIPS
0,Andrews,C,11,22269,48003
1,Angelina,J,23,90437,48005
2,Aransas,N,29,27699,48007
3,Archer,F,13,8344,48009
4,Armstrong,E,33,1948,48011


In [5]:
covid19_df = pd.merge(codes_df, data_file_df,  on="County Name")
covid19_df

,County Name,Section,Number,Population,FIPS,03-09-2020,03-10-2020,03-11-2020,03-12-2020,03-13-2020,...,01-11-2021,01-12-2021,01-13-2021,01-14-2021,01-15-2021,01-16-2021,01-17-2021,01-18-2021,01-19-2021,01-20-2021
0,Andrews,C,11,22269,48003,0,0,0,0,0,...,1523,1558,1558,1558,1558,1606,1606,1606,1606,1606
1,Angelina,J,23,90437,48005,0,0,0,0,0,...,3851,3851,3851,4012,4012,4012,4012,4012,4129,4129
2,Aransas,N,29,27699,48007,0,0,0,0,0,...,607,615,628,636,652,654,665,665,666,678
3,Archer,F,13,8344,48009,0,0,0,0,0,...,574,599,602,605,612,617,617,617,617,626
4,Armstrong,E,33,1948,48011,0,0,0,0,0,...,82,82,83,83,85,85,85,85,86,90
5,Anderson,J,11,58199,48001,0,0,0,0,0,...,4508,4445,4520,4467,4484,4485,4511,4511,4511,4582
6,Atascosa,N,14,51831,48013,0,0,0,0,0,...,2666,2662,2855,2855,2886,2951,2951,2951,2951,3039
7,Austin,K,12,30402,48015,0,0,0,0,0,...,1106,1124,1155,1164,1182,1191,1209,1224,1243,1270
8,Bailey,D,11,7692,48017,0,0,0,0,0,...,550,552,552,554,555,556,556,556,557,557
9,Bandera,B,25,21246,48019,0,0,0,0,0,...,585,588,626,626,633,643,643,643,643,666


In [6]:
# create a parallel dataframe with rates per 10000 people in county
covid19rate_df = covid19_df.copy()

ylim, xlim = covid19rate_df.shape

for y in range(0, ylim):
    pop10000 = covid19rate_df.iloc[y,3]/10000
    for x in range(5, xlim):
        covid19rate_df.iloc[y,x] = covid19rate_df.iloc[y,x]/pop10000

In [7]:
 covid19rate_df

,County Name,Section,Number,Population,FIPS,03-09-2020,03-10-2020,03-11-2020,03-12-2020,03-13-2020,...,01-11-2021,01-12-2021,01-13-2021,01-14-2021,01-15-2021,01-16-2021,01-17-2021,01-18-2021,01-19-2021,01-20-2021
0,Andrews,C,11,22269,48003,0.0,0.0,0.0,0.0,0.0,...,683.910369,699.627285,699.627285,699.627285,699.627285,721.181912,721.181912,721.181912,721.181912,721.181912
1,Angelina,J,23,90437,48005,0.0,0.0,0.0,0.0,0.0,...,425.821290,425.821290,425.821290,443.623738,443.623738,443.623738,443.623738,443.623738,456.560921,456.560921
2,Aransas,N,29,27699,48007,0.0,0.0,0.0,0.0,0.0,...,219.141485,222.029676,226.722986,229.611177,235.387559,236.109607,240.080869,240.080869,240.441893,244.774180
3,Archer,F,13,8344,48009,0.0,0.0,0.0,0.0,0.0,...,687.919463,717.881112,721.476510,725.071908,733.461170,739.453500,739.453500,739.453500,739.453500,750.239693
4,Armstrong,E,33,1948,48011,0.0,0.0,0.0,0.0,0.0,...,420.944559,420.944559,426.078029,426.078029,436.344969,436.344969,436.344969,436.344969,441.478439,462.012320
5,Anderson,J,11,58199,48001,0.0,0.0,0.0,0.0,0.0,...,774.583756,763.758827,776.645647,767.538961,770.459974,770.631798,775.099229,775.099229,775.099229,787.298751
6,Atascosa,N,14,51831,48013,0.0,0.0,0.0,0.0,0.0,...,514.363991,513.592252,550.828655,550.828655,556.809631,569.350389,569.350389,569.350389,569.350389,586.328645
7,Austin,K,12,30402,48015,0.0,0.0,0.0,0.0,0.0,...,363.791856,369.712519,379.909216,382.869548,388.790211,391.750543,397.671206,402.605092,408.854681,417.735675
8,Bailey,D,11,7692,48017,0.0,0.0,0.0,0.0,0.0,...,715.028601,717.628705,717.628705,720.228809,721.528861,722.828913,722.828913,722.828913,724.128965,724.128965
9,Bandera,B,25,21246,48019,0.0,0.0,0.0,0.0,0.0,...,275.345947,276.757978,294.643698,294.643698,297.938435,302.645204,302.645204,302.645204,302.645204,313.470771


In [8]:
# create a parallel dataframe showing each day's increment of cases per county
covid19incr_df = covid19_df.copy()
ylim, xlim = covid19incr_df.shape

for y in range(0, ylim):
    for x in range(6, xlim):
        covid19incr_df.iloc[y,x] = covid19_df.iloc[y,x] - covid19_df.iloc[y,x-1]

In [9]:
# compute the seven-day rolling average of those increments

covid19ravg_df = covid19_df.copy()
ylim, xlim = covid19ravg_df.shape

for y in range(0, ylim):
    pop100000 = covid19rate_df.iloc[y,3]/100000
    for x in range(11, xlim):
        covid19ravg_df.iloc[y,x] = ((covid19incr_df.iloc[y,x] + covid19incr_df.iloc[y,x-1] +
                                    covid19incr_df.iloc[y,x-2] + covid19incr_df.iloc[y,x-3] +
                                    covid19incr_df.iloc[y,x-4] + covid19incr_df.iloc[y,x-5] +
                                    covid19incr_df.iloc[y,x-6]) /  7) / pop100000

In [10]:
# these days come before a seven-day rolling average can be computed, and so are not wanted

del covid19ravg_df["03-09-2020"]
del covid19ravg_df["03-10-2020"]
del covid19ravg_df["03-11-2020"]
del covid19ravg_df["03-12-2020"]
del covid19ravg_df["03-13-2020"]
del covid19ravg_df["03-15-2020"]  # 3-14 is also missing.

covid19ravg_df

,County Name,Section,Number,Population,FIPS,03-16-2020,03-17-2020,03-18-2020,03-19-2020,03-20-2020,...,01-11-2021,01-12-2021,01-13-2021,01-14-2021,01-15-2021,01-16-2021,01-17-2021,01-18-2021,01-19-2021,01-20-2021
0,Andrews,C,11,22269,48003,0.000000,0.000000,0.000000,0.000000,0.000000,...,57.735609,76.980813,76.980813,46.188488,46.188488,53.245062,53.245062,53.245062,30.792325,30.792325
1,Angelina,J,23,90437,48005,0.000000,0.000000,0.000000,0.000000,0.000000,...,36.173564,25.116142,25.116142,50.548211,35.067821,25.432069,25.432069,25.432069,43.913758,43.913758
2,Aransas,N,29,27699,48007,0.000000,0.000000,0.000000,0.000000,0.000000,...,19.598438,19.598438,25.271670,24.240174,28.881909,28.881909,29.913406,29.913406,26.303167,25.787419
3,Archer,F,13,8344,48009,0.000000,0.000000,0.000000,0.000000,0.000000,...,130.119162,107.861937,99.301466,68.483769,66.771675,73.620052,73.620052,73.620052,30.817696,41.090262
4,Armstrong,E,33,1948,48011,0.000000,0.000000,0.000000,0.000000,0.000000,...,36.667644,29.334116,29.334116,22.000587,29.334116,22.000587,22.000587,22.000587,29.334116,51.334702
5,Anderson,J,11,58199,48001,0.000000,0.000000,0.000000,0.000000,0.000000,...,136.968480,19.146132,11.782235,-0.981853,3.191022,0.000000,0.736390,0.736390,16.200573,15.218720
6,Atascosa,N,14,51831,48013,0.000000,0.000000,0.000000,0.000000,0.000000,...,123.202606,57.880419,109.697175,68.078398,66.700292,78.551997,78.551997,78.551997,79.654481,50.714272
7,Austin,K,12,30402,48015,0.000000,0.000000,0.000000,0.000000,0.000000,...,46.519496,48.868965,57.327056,56.387268,62.495888,58.266843,57.327056,55.447480,55.917374,54.037798
8,Bailey,D,11,7692,48017,0.000000,0.000000,0.000000,0.000000,0.000000,...,13.000520,13.000520,9.286086,11.143303,9.286086,11.143303,11.143303,11.143303,9.286086,9.286086
9,Bandera,B,25,21246,48019,0.000000,0.000000,0.000000,0.000000,0.000000,...,104.221299,55.136429,65.894757,43.705706,41.016124,38.998938,38.998938,38.998938,36.981751,26.895819


In [11]:
# assign grades to rates of occurance

covid19grad_df = covid19ravg_df.copy()
ylim, xlim = covid19grad_df.shape

for y in range(0, ylim):
    for x in range(5, xlim):
#        if (covid19grad_df.iloc[y,x] >= 20):
#            covid19grad_df.iloc[y,x] = "A"
#        elif (covid19grad_df.iloc[y,x] >= 10):
#            covid19grad_df.iloc[y,x] = "B"
#        elif (covid19grad_df.iloc[y,x] >= 8):
#            covid19grad_df.iloc[y,x] = "C"
#        elif (covid19grad_df.iloc[y,x] >= 6):
#            covid19grad_df.iloc[y,x] = "D"
#        elif (covid19grad_df.iloc[y,x] >= 4):
#            covid19grad_df.iloc[y,x] = "E"
#        elif (covid19grad_df.iloc[y,x] >= 2):
#            covid19grad_df.iloc[y,x] = "F"
#        elif (covid19grad_df.iloc[y,x] > 0):
#            covid19grad_df.iloc[y,x] = "G";
#        else: covid19grad_df.iloc[y,x] = " ";
        covid19grad_df.iloc[y,x] = assign_grade(covid19grad_df.iloc[y,x])
            
covid19grad_df
            

,County Name,Section,Number,Population,FIPS,03-16-2020,03-17-2020,03-18-2020,03-19-2020,03-20-2020,...,01-11-2021,01-12-2021,01-13-2021,01-14-2021,01-15-2021,01-16-2021,01-17-2021,01-18-2021,01-19-2021,01-20-2021
0,Andrews,C,11,22269,48003,none,none,none,none,none,...,>40,>40,>40,>40,>40,>40,>40,>40,>28,>28
1,Angelina,J,23,90437,48005,none,none,none,none,none,...,>28,>21,>21,>40,>28,>21,>21,>21,>40,>40
2,Aransas,N,29,27699,48007,none,none,none,none,none,...,>15,>15,>21,>21,>28,>28,>28,>28,>21,>21
3,Archer,F,13,8344,48009,none,none,none,none,none,...,>40,>40,>40,>40,>40,>40,>40,>40,>28,>40
4,Armstrong,E,33,1948,48011,none,none,none,none,none,...,>28,>28,>28,>21,>28,>21,>21,>21,>28,>40
5,Anderson,J,11,58199,48001,none,none,none,none,none,...,>40,>15,>10,negative,>0,none,>0,>0,>15,>15
6,Atascosa,N,14,51831,48013,none,none,none,none,none,...,>40,>40,>40,>40,>40,>40,>40,>40,>40,>40
7,Austin,K,12,30402,48015,none,none,none,none,none,...,>40,>40,>40,>40,>40,>40,>40,>40,>40,>40
8,Bailey,D,11,7692,48017,none,none,none,none,none,...,>10,>10,>5,>10,>5,>10,>10,>10,>5,>5
9,Bandera,B,25,21246,48019,none,none,none,none,none,...,>40,>40,>40,>40,>40,>28,>28,>28,>28,>21


In [12]:
covid19rate_df.to_csv("covid19rate.csv", index=False, header=True)
covid19ravg_df.to_csv("covid19ravg.csv", index=False, header=True)
covid19grad_df.to_csv("covid19grad.csv", index=False, header=True)
covid19incr_df.to_csv("covid19incr.csv", index=False, header=True)

In [13]:
# This puts the data in a format that's friendlier to Tableau

counties = []
populations = []
dates = []
values = []
fips = []
grades = []
weeks = []
columns = covid19ravg_df.columns

ylim, xlim = covid19ravg_df.shape

for y in range(0, ylim):
    cty = covid19ravg_df.iloc[y,0]
    pop = covid19ravg_df.iloc[y,3]
    fip = covid19ravg_df.iloc[y,4]
    ctr = 99
    for x in range(5, xlim):
        ctr = ctr + 1
        if ctr > 7:
            ctr = 1
        week = ctr
        counties.append(cty)
        populations.append(pop)
        fips.append(fip)
        dates.append(columns[x])
        weeks.append(week)
        values.append(covid19ravg_df.iloc[y,x])
        grades.append(assign_grade(covid19ravg_df.iloc[y,x]))
        
covid19ravg2_df = pd.DataFrame({
    "Counties": counties,
    "Populations": populations,
    "FIPS": fips,
    "Dates": dates,
    "Weeks": weeks,
    "Values": values,
    "Grades": grades
    })

covid19ravg2_df

,Counties,Populations,FIPS,Dates,Weeks,Values,Grades
0,Andrews,22269,48003,03-16-2020,1,0.000000,none
1,Andrews,22269,48003,03-17-2020,2,0.000000,none
2,Andrews,22269,48003,03-18-2020,3,0.000000,none
3,Andrews,22269,48003,03-19-2020,4,0.000000,none
4,Andrews,22269,48003,03-20-2020,5,0.000000,none
5,Andrews,22269,48003,03-21-2020,6,0.000000,none
6,Andrews,22269,48003,03-22-2020,7,0.000000,none
7,Andrews,22269,48003,03-23-2020,1,0.000000,none
8,Andrews,22269,48003,03-24-2020,2,0.000000,none
9,Andrews,22269,48003,03-25-2020,3,0.000000,none


In [14]:
covid19ravg2_df.to_csv("covid19ravg2.csv", index=False, header=True)

In [ ]:
data_file_df = pd.read_excel('Texas COVID-19 Case Count Data by County.xlsx', skiprows={0, 1, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267})

cols = data_file_df.columns.values
newcols = []

for col in cols:
    if col == 'County Name':
        newcols.append(col)
    else:
        newcol = ""

        for char in col:
            if (char >= "0" and char <= "9") or char == "-":
                newcol = newcol + char
        print('/' + newcol + "/")
        newcols.append(newcol)

data_file_df.columns = newcols

In [ ]:
# data_file_df = pd.read_excel('Texas COVID-19 Case Count Data by County.xlsx', skiprows={0, 1, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267})
data_file_df.columns.values

In [ ]:
covid19ravg_df.shape